In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# About nn.Module class

In [2]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_feats = 1
        for s in size:
            num_feats *= s
        return num_feats

In [3]:
net = ConvNet()
print(net)

# params = list(net.parameters())
# print(len(params))
# for i in range(len(params)):
#     print(params[i].size())

# for param in net.parameters():
#     print(type(param))

ConvNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## nn.Module attribute

In [11]:
class Sub(nn.Module):
    
    def __init__(self):
        super(Sub, self).__init__()
        
        self.fc1 = nn.Linear(3, 7)
    
    def forward(self, x):
        out = self.fc1(x)
        
        return out

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(3, 5)
        self.fc2 = nn.Linear(5, 3)
        self.fc3 = Sub()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

net = Net()
print("===> Net architecture:")
print(net)

print("===> buffers:")
for buf in net.buffers():
    print("---")
    print(type(buf), buf.size())

print("===> children:")
for chd in net.children():
    print(type(chd), chd)

print("===> named_children:")
for n, n_chd in net.named_children():
    print(n, f'-> ({type(n)}), ', n_chd)
    
print("===> modules:")
for modu in net.modules():
    print(modu)

print("===> parameters:")
for para in net.parameters():
    print(para)
    
print("===> named_parameters:")
init_flag = True
for n, para in net.named_parameters(recurse=True):
    print(n, f'-> ({type(n)})')
    if init_flag:
        if n.endswith(".bias"):
            nn.init.constant_(para, 0.0)
    print(para)

# bad impelement
print("===> initial test:")
for chd in net.children():
    print(isinstance(chd, nn.Linear))
    if isinstance(chd, nn.Linear):
        for para_name, para in chd.named_parameters():
            print(para_name)
            if para_name.endswith(".weight"):
                nn.init.constant_(m.weight, 0.0)
                print(m.weight)
            if para_name.endswith(".bias"):
                nn.init.constant_(m.bias, 0.0)

===> Net architecture:
Net(
  (fc1): Linear(in_features=3, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=3, bias=True)
  (fc3): Sub(
    (fc1): Linear(in_features=3, out_features=7, bias=True)
  )
)
===> buffers:
===> children:
<class 'torch.nn.modules.linear.Linear'> Linear(in_features=3, out_features=5, bias=True)
<class 'torch.nn.modules.linear.Linear'> Linear(in_features=5, out_features=3, bias=True)
<class '__main__.Sub'> Sub(
  (fc1): Linear(in_features=3, out_features=7, bias=True)
)
===> named_children:
fc1 -> (<class 'str'>),  Linear(in_features=3, out_features=5, bias=True)
fc2 -> (<class 'str'>),  Linear(in_features=5, out_features=3, bias=True)
fc3 -> (<class 'str'>),  Sub(
  (fc1): Linear(in_features=3, out_features=7, bias=True)
)
===> modules:
Net(
  (fc1): Linear(in_features=3, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=3, bias=True)
  (fc3): Sub(
    (fc1): Linear(in_features=3, out_features=7, bias=True)
  )
)
Linea

## initialization

In [25]:
class Neti(nn.Module):
    
    def __init__(self):
        super(Neti, self).__init__()
        
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2, bias=False)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        
        return out

def _init_weight(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)


neti = Neti()

print("===> hasattr:")
for chld_name, chld in neti.named_children():
        print(f'{chld_name} -->', hasattr(chld, "bias"))
        
print("===> apply fn:")
neti.apply(_init_weight)
for para in neti.parameters():
    print(para)

===> hasattr:
fc1 --> True
fc2 --> True
===> apply fn:
Parameter containing:
tensor([[-0.2439,  0.4455],
        [-0.5618, -0.2808]], requires_grad=True)
Parameter containing:
tensor([0., 0.], requires_grad=True)
Parameter containing:
tensor([[-0.0714, -0.1229],
        [-0.3134, -0.4178]], requires_grad=True)


# About Loss

In [4]:
inpt = torch.randn(1, 1, 32, 32)
out = net(inpt)
print(out)
y = torch.randn(10).view(1, -1)
print(y)

res = torch.mean(torch.pow(out-y, 2))
print(res)

criterion = nn.MSELoss()
loss = criterion(out, y)
print(loss)

tensor([[ 0.0113,  0.0475, -0.0518,  0.0591,  0.0167,  0.0898,  0.1361, -0.1081,
          0.0605,  0.0512]], grad_fn=<AddmmBackward>)
tensor([[-1.1377, -0.5150, -1.2655,  0.6322,  0.5224, -2.1549, -0.8694,  0.9398,
          1.4885,  0.6093]])
tensor(1.3192, grad_fn=<MeanBackward0>)
tensor(1.3192, grad_fn=<MseLossBackward>)
